In [3]:
import pandas as pd
import numpy as np
import math
epsilon = 0.000000001

In [4]:
dataset = pd.read_excel("slides.xlsx")

In [5]:
dataset

,ex,Alt,Bar,Fri,Hun,Pat,Price,Rain,Res,Type,Est,Goal
0,x1,Yes,No,No,Yes,Some,$$$,No,Yes,French,"""0 - 10""",1
1,x2,Yes,No,No,Yes,Full,$,No,No,Thai,"""30 - 60""",0
2,x3,No,Yes,No,No,Some,$,No,No,Burger,"""0 - 10""",1
3,x4,Yes,No,Yes,Yes,Full,$,Yes,No,Thai,"""10 - 30""",1
4,x5,Yes,No,Yes,No,Full,$$$,No,Yes,French,"""> 60""",0
5,x6,No,Yes,No,Yes,Some,$$,Yes,Yes,Italian,"""0 - 10""",1
6,x7,No,Yes,No,No,None,$,Yes,No,Burger,"""0 - 10""",0
7,x8,No,No,No,Yes,Some,$$,Yes,Yes,Thai,"""0 - 10""",1
8,x9,No,Yes,Yes,No,Full,$,Yes,No,Burger,"""> 60""",0
9,x10,Yes,Yes,Yes,Yes,Full,$$$,No,Yes,Italian,"""10 - 30""",0


In [93]:
class node ():
    def __init__(self, gainInfo = None , label  = None, feature = None , right = None , left = None , type = None) :
        self.children = []
        self.feature = feature
        self.type = type
        self.geinInfo = gainInfo
        self.label = label

In [8]:
def MakeUnique(dataFrame , columnName):
    #print(dataFrame[columnName])
    PossibleValues = pd.unique(dataFrame[columnName])
    #print(PossibleValues)
    indexed = {}
    for count , value in enumerate(PossibleValues):
        indexed[value] = count
    for i in range(len( dataFrame[columnName])):
        dataFrame.loc[i , columnName] =  indexed[dataFrame[columnName][i]] 

In [9]:
def MakeUniqueInt (dataFrame , columnName):
    maxvalue = np.max(dataFrame[columnName])
    minvalue = np.min(dataFrame[columnName])
    lenghtEachSlice = (maxvalue - minvalue + 1 ) / len(dataFrame[columnName])
    indexed = {}
    for count , value in enumerate( range(len(dataFrame[columnName] / lenghtEachSlice)) ):
        indexed[(value * lenghtEachSlice + minvalue , (value + 1) * lenghtEachSlice + minvalue)] = count
    for i in range(len(dataFrame[columnName])):
        temp = dataFrame.loc[i , columnName]
        flag = False
        for j in indexed :
            if temp >= j[0] and  temp < j[1]:
                flag = True
                dataFrame.loc[i , columnName] = indexed[j]
        if flag == False:
            if temp < indexed[0][0] :
                dataFrame.loc[i , columnName] = -1
            else:
                dataFrame.loc[i , columnName] = len(indexed) + 1

In [10]:
def plurality_value(example):
    counter = 0
    for i in range(len(example)) :
        if example.iloc[i , -1] == 1 :
            counter = counter + 1
    if counter >= example.shape[0] / 2:
        return 1 , counter
    return 0 , counter

In [11]:
def entropy (example):
    temp , amount = plurality_value(example)
    probability = amount / example.shape[0]
    if probability == 0 :
        return -1 * ( probability * math.log2(probability + epsilon) + (1 - probability) * math.log2(1 - probability))
    if probability == 1 :
        return -1 * ( probability * math.log2(probability) + (1 - probability) * math.log2(1 - probability + epsilon))
    return -1 * ( probability * math.log2(probability) + (1 - probability) * math.log2(1 - probability))

test the MakeUnique

In [12]:
MakeUnique(dataset , "Type")

In [13]:
dataset

,ex,Alt,Bar,Fri,Hun,Pat,Price,Rain,Res,Type,Est,Goal
0,x1,Yes,No,No,Yes,Some,$$$,No,Yes,0,"""0 - 10""",1
1,x2,Yes,No,No,Yes,Full,$,No,No,1,"""30 - 60""",0
2,x3,No,Yes,No,No,Some,$,No,No,2,"""0 - 10""",1
3,x4,Yes,No,Yes,Yes,Full,$,Yes,No,1,"""10 - 30""",1
4,x5,Yes,No,Yes,No,Full,$$$,No,Yes,0,"""> 60""",0
5,x6,No,Yes,No,Yes,Some,$$,Yes,Yes,3,"""0 - 10""",1
6,x7,No,Yes,No,No,None,$,Yes,No,2,"""0 - 10""",0
7,x8,No,No,No,Yes,Some,$$,Yes,Yes,1,"""0 - 10""",1
8,x9,No,Yes,Yes,No,Full,$,Yes,No,2,"""> 60""",0
9,x10,Yes,Yes,Yes,Yes,Full,$$$,No,Yes,3,"""10 - 30""",0


In [14]:
type(dataset["Fri"][1])

str

In [43]:
datasetNparr = np.array(dataset)

test the make unique for ints

In [15]:
intlist = [12 , 1 , 123, 23, 22, 5, 77]
dftest = pd.DataFrame(intlist , columns=["test"])


In [16]:
MakeUniqueInt(dftest , "test")

In [17]:
dftest

,test
0,0
1,0
2,6
3,1
4,1
5,0
6,4


test the plurality value

In [18]:
plurality_value(dataset)

(1, 6)

test entropy 

In [19]:
entropy(dataset)

1.0

In [20]:
def Remainder(dataset , feature):
    summation = 0
    temp , p = plurality_value(dataset)
    n = dataset.shape[0] - p
    values = np.unique(dataset[feature])
    # print(values)
    for value in values:
        # print(value)
        temp , counter = plurality_value(dataset.loc[dataset[feature] == value]) 
        pk = counter
        nk = len(dataset.loc[dataset[feature] == value]) - pk
        # print(f"value is{value} and pk is{pk}")
        # print(summation)
        summation = summation + (((pk + nk) / ( p + n)) * entropy(dataset.loc[dataset[feature] == value]))
    return abs(summation)
                

In [54]:
def Gain(example , feature):
    return entropy(example) - Remainder(example , feature)

In [21]:
dataset.loc[dataset["Goal"] == 1]

,ex,Alt,Bar,Fri,Hun,Pat,Price,Rain,Res,Type,Est,Goal
0,x1,Yes,No,No,Yes,Some,$$$,No,Yes,0,"""0 - 10""",1
2,x3,No,Yes,No,No,Some,$,No,No,2,"""0 - 10""",1
3,x4,Yes,No,Yes,Yes,Full,$,Yes,No,1,"""10 - 30""",1
5,x6,No,Yes,No,Yes,Some,$$,Yes,Yes,3,"""0 - 10""",1
7,x8,No,No,No,Yes,Some,$$,Yes,Yes,1,"""0 - 10""",1
11,x12,Yes,Yes,Yes,Yes,Full,$,No,No,2,"""30 - 60""",1


In [24]:
Remainder(dataset , "Pat")

0.4591479170272448

In [22]:
dataset

,ex,Alt,Bar,Fri,Hun,Pat,Price,Rain,Res,Type,Est,Goal
0,x1,Yes,No,No,Yes,Some,$$$,No,Yes,0,"""0 - 10""",1
1,x2,Yes,No,No,Yes,Full,$,No,No,1,"""30 - 60""",0
2,x3,No,Yes,No,No,Some,$,No,No,2,"""0 - 10""",1
3,x4,Yes,No,Yes,Yes,Full,$,Yes,No,1,"""10 - 30""",1
4,x5,Yes,No,Yes,No,Full,$$$,No,Yes,0,"""> 60""",0
5,x6,No,Yes,No,Yes,Some,$$,Yes,Yes,3,"""0 - 10""",1
6,x7,No,Yes,No,No,None,$,Yes,No,2,"""0 - 10""",0
7,x8,No,No,No,Yes,Some,$$,Yes,Yes,1,"""0 - 10""",1
8,x9,No,Yes,Yes,No,Full,$,Yes,No,2,"""> 60""",0
9,x10,Yes,Yes,Yes,Yes,Full,$$$,No,Yes,3,"""10 - 30""",0


In [50]:
def CompleteUnique(dataset, datasetNparr):
    for i in range (dataset.shape[1]):
        if type(datasetNparr[i]) == int :
            MakeUniqueInt(dataset , dataset.columns[i])
        else :
            MakeUnique(dataset ,  dataset.columns[i])

In [29]:
dataset.head()

,ex,Alt,Bar,Fri,Hun,Pat,Price,Rain,Res,Type,Est,Goal
0,x1,Yes,No,No,Yes,Some,$$$,No,Yes,0,"""0 - 10""",1
1,x2,Yes,No,No,Yes,Full,$,No,No,1,"""30 - 60""",0
2,x3,No,Yes,No,No,Some,$,No,No,2,"""0 - 10""",1
3,x4,Yes,No,Yes,Yes,Full,$,Yes,No,1,"""10 - 30""",1
4,x5,Yes,No,Yes,No,Full,$$$,No,Yes,0,"""> 60""",0


In [46]:
dataset.columns[1]

'Alt'

In [36]:
test = np.array(dataset)

***test the CompleteUnique() function***


In [51]:
CompleteUnique(dataset , datasetNparr)

In [53]:
dataset

,ex,Alt,Bar,Fri,Hun,Pat,Price,Rain,Res,Type,Est,Goal
0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,1,1,0,1,1,1,1
2,2,1,1,0,1,0,1,0,1,2,0,0
3,3,0,0,1,0,1,1,1,1,1,2,0
4,4,0,0,1,1,1,0,0,0,0,3,1
5,5,1,1,0,0,0,2,1,0,3,0,0
6,6,1,1,0,1,2,1,1,1,2,0,1
7,7,1,0,0,0,0,2,1,0,1,0,0
8,8,1,1,1,1,1,1,1,1,2,3,1
9,9,0,1,1,0,1,0,0,0,3,2,1


In [70]:
def sameClassification(example):
    temp = example[0 , -1]
    for i in range(example.shape[0]):
        if temp != example.iloc[i , -1]:
            return False
    return True

In [69]:
dataset.iloc[0 , -1]

0

In [64]:
sameClassification(dataset)

0      0
1      1
2      2
3      3
4      4
5      5
6      6
7      7
8      8
9      9
10    10
11    11
Name: ex, dtype: object
0     0
1     0
2     1
3     0
4     0
5     1
6     1
7     1
8     1
9     0
10    1
11    0
Name: Alt, dtype: object
0     0
1     0
2     1
3     0
4     0
5     1
6     1
7     0
8     1
9     1
10    0
11    1
Name: Bar, dtype: object
0     0
1     0
2     0
3     1
4     1
5     0
6     0
7     0
8     1
9     1
10    0
11    1
Name: Fri, dtype: object
0     0
1     0
2     1
3     0
4     1
5     0
6     1
7     0
8     1
9     0
10    1
11    0
Name: Hun, dtype: object
0     0
1     1
2     0
3     1
4     1
5     0
6     2
7     0
8     1
9     1
10    2
11    1
Name: Pat, dtype: object
0     0
1     1
2     1
3     1
4     0
5     2
6     1
7     2
8     1
9     0
10    1
11    1
Name: Price, dtype: object
0     0
1     0
2     0
3     1
4     0
5     1
6     1
7     1
8     1
9     0
10    0
11    0
Name: Rain, dtype: object
0     0
1     1
2 

In [7]:
class DecisionTree():
    def __init__(self):
        self.root = None
    def build (self, example , features , parent_examples):
        if example.empty:
            return plurality_value(parent_examples)
        elif sameClassification(example) :
            return plurality_value(example)[0]
        elif len(features) == 0 : 
            return plurality_value(example)[0]
        else :
            temp = features[0]
            maxGain = 0 ;
            for i in features :
                if Gain(i) > maxGain :
                    maxGain = Gain(i)
                    temp = i
            new = DecisionTree( node( maxGain , feature= temp ))
            values = np.unique(dataset[temp])
            for j in values:
                exs = dataset[dataset[temp] == j]
                sub = DecisionTree( node( maxGain , feature=temp ) )
                sub.build(exs , features.remove(temp), example)
                self.children.append(sub)
        return self
                
        